# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [0]:
import keras
from keras.datasets import cifar10

In [0]:
(x_train,y_train), (x_test,y_test) = cifar10.load_data()

In [0]:
print(x_train.shape)
print(y_train.shape)

In [0]:
import numpy as np
y_train = y_train.reshape(y_train.shape[0])
y_test = y_test.reshape(y_test.shape[0])

In [0]:
x_train_lt5 = x_train[y_train<5]
y_train_lt5 = y_train[y_train<5]
x_test_lt5 = x_test[y_test<5]
y_test_lt5 = y_test[y_test<5]

x_train_gt5 = x_train[y_train>=5]
y_train_gt5 = y_train[y_train>=5]
x_test_gt5 = x_test[y_test>=5]
y_test_gt5 = y_test[y_test>=5]

### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
import pandas as pd
y_train_lt5 = pd.get_dummies(y_train_lt5)
y_test_lt5 = pd.get_dummies(y_test_lt5)

y_train_gt5 = pd.get_dummies(y_train_gt5)
y_test_gt5 = pd.get_dummies(y_test_gt5)


In [0]:
print(x_train_lt5.shape)
print(y_train_lt5.shape)
print(x_test_lt5.shape)
print(y_test_lt5.shape)

In [0]:
print(x_train_gt5.shape)
print(y_train_gt5.shape)
print(x_test_gt5.shape)
print(y_test_gt5.shape)

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [0]:
from keras import Sequential
from keras.layers import Flatten,BatchNormalization,MaxPooling2D,Dropout, Convolution2D, Dense

In [0]:
model = Sequential()
model.add(BatchNormalization(input_shape = (32,32,3)))
model.add(Convolution2D(32,(3,3), activation = 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.2))
model.add(Convolution2D(64,(3,3), activation = 'relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.2))

model.add(Flatten())
#model.add(BatchNormalization())
model.add(Dense(128,activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(5,activation = 'softmax'))

In [0]:
model.summary()

In [0]:
from keras.optimizers import adam
optimizer = adam(lr = 0.001)
model.compile(optimizer = optimizer, loss = 'categorical_crossentropy',metrics = ['accuracy'])
model.fit(x_train_lt5,y_train_lt5, validation_data=(x_test_lt5,y_test_lt5), epochs = 30, batch_size= 32) 

In [0]:
model.evaluate(x_test_lt5,y_test_lt5)

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [0]:
for layers in model.layers:
    print(layers.name)
    if('dense' not in layers.name):
        layers.trainable = False

In [0]:
#Module to print colourful statements
from termcolor import colored

#Check which layers have been frozen 
for layer in model.layers:
  print (colored(layer.name, 'blue'))
  print (colored(layer.trainable, 'red'))

### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [0]:
from keras.optimizers import adam
optimizer = adam(lr = 0.001)
model.compile(optimizer = optimizer, loss = 'categorical_crossentropy',metrics = ['accuracy'])
model.fit(x_train_gt5,y_train_gt5,validation_data=(x_test_gt5,y_test_gt5), epochs = 10, batch_size= 32)

In [0]:
model.evaluate(x_test_gt5,y_test_gt5)

# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [0]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [0]:
twenty_train.target

In [0]:
twenty_train.target_names

In [0]:
twenty_train.data[0:5]

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=0, max_df=1.0, ngram_range=(1,1))
train_tfidf_features = vectorizer.fit_transform(twenty_train.data).astype(float)

In [0]:
vectorizer.vocabulary_

In [0]:
test_tfidf_features = vectorizer.transform(twenty_test.data).astype(float)

In [0]:
twenty_test.target

### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [0]:
# use logistic regression with text column only
logreg = LogisticRegression()
logreg.fit(train_tfidf_features, twenty_train.target)
y_pred_class = logreg.predict(test_tfidf_features)
print(metrics.accuracy_score(y_pred_class, twenty_test.target))

In [0]:
y_train_class = logreg.predict(train_tfidf_features)

In [0]:
print(metrics.accuracy_score(y_train_class, twenty_train.target))